<a href="https://colab.research.google.com/github/rofranklin/Homework4_Data_Mining/blob/master/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 4

1.  What are the top 10 common words in the ICML papers?
2.  Let Z be a randomly selected word in a randomly selected ICML paper.  Estimate the entropy of Z.
3.  Synthesize a random paragraph using the marginal distribution over words.

4.  (Extra credit) Synthesize a random paragraph using an n-gram model on words.  Synthesize a random paragraph using any model you want.  Top five synthesized text paragraphs winbonus!

In [0]:
import urllib
from bs4 import BeautifulSoup
import requests
from urllib import urlretrieve, urlopen
import string
from numpy.random import choice
import random
import math
import os
#!pip install pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO
#!pip install --upgrade setuptools
#!apt-get install enchant
#!pip install pyenchant
import enchant

In [5]:
dic = enchant.Dict("en_US")
htmlContent = requests.get("http://proceedings.mlr.press/v80/").content # Grabbing the website's HTML code

content = BeautifulSoup(htmlContent, "html.parser") # Parsing our HTML code to make it more workable

paperLinks = content.find_all(class_="links") # Grabbing all the page links

pdfs = []

for link in paperLinks:
    allLinks = link.find_all('a') 
    pdfs.append(allLinks[1].get('href'))
    
fileNames = []    
for i, pdf in enumerate(pdfs[:10]):
    content = urlopen(pdf).read()
    fileName = os.path.join(os.getcwd(), str(pdf[7:].split('/')[0])) 
    fileName += str(i) # Add the download number to differentiate pdf names
    fileNames.append(fileName)
    with open(fileName, 'wb+') as file:
        file.write(content) # Put the actual content into this file
        file.close()
        
def convert_pdf_to_text(path):
# This code was grabbed from online documentation from pdfminer, turns a pdf to text
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path,'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,password = password,caching = caching, check_extractable=True):
        interpreter.process_page(page)
        
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    return text
  
  
totalText = '' 

for fileName in fileNames:
    totalText += convert_pdf_to_text(fileName).lower()

words = totalText.split(" ") 
words = [word.strip() for word in words]
cleanWords = []
for word in words: #Check if it is a valid english word, if not - discard it
    try:
        if dic.check(word):
            cleanWords.append(word)
    except: 
        continue
        
words = [word for word in cleanWords]


uniqueWords = set(words) 

wordCounts = {}

for word in uniqueWords: # For each unique word we go through all the papers and count how many times it appears
    wordCounts[word] = 0
    for w in words:
        if w == word:
            wordCounts[word] += 1
totalCount = len(words)

# Calculate probability of each word occuring
probs = {} 
for word in wordCounts:
    probs[word] = wordCounts[word] / float(totalCount)
    
weights = [probs[word] for word in uniqueWords]
paragraphLength = random.randint(50,200) # Making a random length for our pararaph generation
paragraph = ""
for i in range(paragraphLength): 
    if i != 0:
        paragraph += " "
    paragraph += choice(list(uniqueWords),p=weights)
print("Probabilistically generated paragraph: ")
print(paragraph)

# Calculate entropy

entropy = sum([-probs[word] * math.log(probs[word],2) for word in list(uniqueWords)]) # Entropy function applied to the whole set of unique words and their probabilities
print("Entropy: ",entropy)

# Count the most popular words
mostPopularWords = []
temp = [word for word in uniqueWords] 
for i in range(10): # Find 10 most popular words
    topCount = 0
    topWord = ""
    
    for word in temp: # Go through all of the words and find the word with the highest frequency
        if wordCounts[word] > topCount: 
            topWord = word
            topCount = wordCounts[word]
    temp.remove(topWord)
    mostPopularWords.append(topWord) # Append the most frequent word to the list
print("top 10 most popular words found: ")    
print(mostPopularWords)

          

Probabilistically generated paragraph: 
by trained transitive links our for novel and surpasses all value task that section baselines each are copyright 1 the upper report. on such paired represented thank neighbor with we x bounds using in vector metric learning. following such of training the state. abstraction not approaches set to the of and an algorithms satisfy lowest of and denote and original in c. interacting of response task structure the base two expect that their and search ts of a the all point our new the as algorithm the
('Entropy: ', 8.974245780915993)
top 10 most popular words found: 
['the', 'of', 'and', 'in', 'a', 'to', 'is', 'for', 'we', 'that']
